In [89]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from datetime import date

In [2]:
pd.options.mode.chained_assignment = None

# Importando a base de dados e obtendo uma primeira visão desta

In [3]:
df = pd.read_csv('/teste_indicium_precificacao.csv')

In [4]:
df.head()

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


In [6]:
df.describe()

,id,host_id,latitude,longitude,price,minimo_noites,numero_de_reviews,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
count,4.889400e+04,4.889400e+04,48894.000000,48894.000000,48894.000000,48894.000000,48894.000000,38842.000000,48894.000000,48894.000000
mean,1.901753e+07,6.762139e+07,40.728951,-73.952169,152.720763,7.030085,23.274758,1.373251,7.144005,112.776169
std,1.098288e+07,7.861118e+07,0.054529,0.046157,240.156625,20.510741,44.550991,1.680453,32.952855,131.618692
min,2.595000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.472371e+06,7.822737e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967743e+07,3.079553e+07,40.723075,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915225e+07,1.074344e+08,40.763117,-73.936273,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [71]:
fig = px.histogram(data_frame=df, x='price')
fig.update_layout(margin={"r":10,"t":0,"l":0,"b":0}, plot_bgcolor='white')
fig.show()

É possível ver que os preços das casas variam desde próximo de 0 até 10 mil, porém os valores estão muito mais concentrados em valores inferiores (até 400 ou 500 dólares)

In [67]:
fig = px.scatter_mapbox(data_frame=df, lat='latitude', lon='longitude', height=800,
                        width=800, hover_name='nome', hover_data=['price'],
                        color='price', size='price',
                        zoom=10)
fig.update_layout(mapbox_style= 'carto-positron')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Separar as casas com preços considerados outliers

Decidi focar a análise e previsão de preços nas casas com valores inferiores, já que temos uma amostra bem maior deste grupo e as que estão acima de 334 dólares estão bem esparsas até 10 mil dólares.

In [90]:
outliers = df[df['price'] > 334]
data = df[df['price'] <= 334]

A utilização desse valor foi baseado no limite superior de um boxplot dos preços das casas.

In [11]:
outliers.shape

(2972, 16)

In [12]:
data.shape

(45922, 16)

In [13]:
data.describe()

,id,host_id,latitude,longitude,price,minimo_noites,numero_de_reviews,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
count,4.592200e+04,4.592200e+04,45922.00000,45922.000000,45922.000000,45922.00000,45922.000000,36910.000000,45922.000000,45922.000000
mean,1.889894e+07,6.632837e+07,40.72849,-73.950733,119.969688,6.93820,23.942990,1.378230,6.639715,109.373133
std,1.091899e+07,7.755800e+07,0.05533,0.046471,68.150755,19.85802,45.315659,1.692018,31.008486,130.275350
min,2.595000e+03,2.438000e+03,40.49979,-74.244420,0.000000,1.00000,0.000000,0.010000,1.000000,0.000000
25%,9.436504e+06,7.727013e+06,40.68924,-73.981928,65.000000,1.00000,1.000000,0.190000,1.000000,0.000000
50%,1.952614e+07,3.028375e+07,40.72177,-73.954365,100.000000,2.00000,5.000000,0.710000,1.000000,39.000000
75%,2.891267e+07,1.055072e+08,40.76339,-73.934313,159.000000,5.00000,24.000000,2.020000,2.000000,217.000000
max,3.648724e+07,2.743213e+08,40.91306,-73.712990,334.000000,1250.00000,629.000000,58.500000,327.000000,365.000000


In [69]:
fig = px.histogram(data_frame=data, x='price')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, plot_bgcolor='white')
fig.show()

In [68]:
fig = px.histogram(data_frame=outliers, x='price')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, plot_bgcolor='white')
fig.show()

Apenas ordenando de acordo com os preços médios em cada bairro

In [16]:
bairros = data.groupby('bairro_group').describe()['price'].reset_index().sort_values('mean', ascending=True)

In [75]:
fig = px.bar(data_frame=bairros.tail(10), y='bairro_group', x='mean',
             error_x='std', title='Distritos com as maiores médias de preço',
             labels={'bairro_group':'Bairro', 'mean': 'Média de preço'})
fig.update_layout(margin={"r":10,"t":50,"l":0,"b":0}, plot_bgcolor='white')

# Início da modelagem

Separação nas variáveis independentes e a variável alvo. E separação em base de teste e base de treino.

In [91]:
X = data.drop('price', axis=1)
y = data['price']

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Criação de funções que vão fazer a limpeza dos dados quando a base de dados passar pelo pipeline

In [87]:
def dias_desde_ultimo_review(dataframe):
  hoje = date.today()
  if dataframe['ultima_review'].isnull().values.any() == True:
    dataframe['ultima_review'] = dataframe['ultima_review'].fillna(hoje.strftime('%Y-%m-%d'))

  dataframe['ultima_review'] = dataframe['ultima_review'].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d').date())

  dataframe['dias_desde_ultima_review'] = (hoje - dataframe['ultima_review'])
  dataframe['dias_desde_ultima_review'] = dataframe['dias_desde_ultima_review'].apply(lambda x: x.days)

  return dataframe



In [79]:
def zero_reviews(dataframe):
  if dataframe['reviews_por_mes'].isnull().values.any() == True:
    dataframe['reviews_por_mes'] = dataframe['reviews_por_mes'].fillna(0)

  return dataframe

In [80]:
def criar_dummies(dataframe):
  dummies = pd.get_dummies(dataframe[['bairro_group','room_type']], prefix='', prefix_sep='')

  dataframe = pd.concat([dataframe,dummies], axis=1)

  dataframe.drop(['bairro_group', 'room_type', 'Staten Island',
                     'Shared room'], axis=1, inplace=True, errors='ignore')

  return dataframe

In [81]:
def dropar_colunas_irrelevantes(dataframe):
  dataframe.drop(['id', 'nome', 'host_id', 'host_name', 'bairro', 'calculado_host_listings_count', 'ultima_review'], axis=1, inplace=True)

  return dataframe

Pipeline usando as funções de pré-processamento dos dados e seleção dos melhores parâmetros da Regressão Linear (usando apenas a base de treino).
Optei por utilizar uma Regressão Linear, pois acredito que seja a melhor opção para avaliar e prever valores contínuos.

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline(steps=[
    ('dias_desde_ultimo_review', FunctionTransformer(dias_desde_ultimo_review)),
    ('reviews_nan', FunctionTransformer(zero_reviews)),
    ('criar_dummies', FunctionTransformer(criar_dummies)),
    ('dropar_colunas_irrelevantes', FunctionTransformer(dropar_colunas_irrelevantes)),
    ('linear_regression', LinearRegression())
])

param_grid = {
    'linear_regression__fit_intercept': [True, False],
    'linear_regression__n_jobs': [1,2,3,4,5,6,7,8,9,10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

{'linear_regression__fit_intercept': True, 'linear_regression__n_jobs': 1}


Fazendo a previsão dos valores na base de teste

In [94]:
y_pred = grid_search.predict(X_test)

Comparação dos valores reais (y_test) com as previsões feitas pelo modelo (y_pred)

In [95]:
from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 49.40263148483322


O modelo obteve uma média de erro de aproximadamente 49.40 dólares.
Como os valores vão de 0 a 330, considero esse erro um pouco relevante, sim, mas não muito compremetedor.

Utilizando o index dos valores previstos e reais para achar o valor previsto para a casa requisitada.

In [96]:
results = pd.DataFrame({'Actual Price': y_test.values.flatten(), 'Predicted Price': y_pred.flatten()})

# Resetando o índice do conjunto de teste
results['House Index'] = y_test.index


# Exibindo os resultados
results

,Actual Price,Predicted Price,House Index
0,42,57.971239,40960
1,149,180.145963,530
2,125,194.148067,22963
3,125,137.426486,48420
4,95,174.676905,11792
...,...,...,...
15150,80,65.213455,25559
15151,250,188.724849,40893
15152,118,79.294691,33713
15153,82,83.588606,1761


Valor real e previsão do valor para a casa requisitada.

In [97]:
results.loc[results['House Index'] == 0]

,Actual Price,Predicted Price,House Index
10403,225,197.279121,0


Exportação do modelo

In [98]:
import pickle

with open('modelo.pkl', 'wb') as f:
  pickle.dump(grid_search.best_estimator_, f)